In [ ]:
import pandas as pd
from sentiment_llm import analyze_review
from dotenv import load_dotenv
load_dotenv()

df = pd.read_csv("reviews.csv")  
df=df[['review', 'sentiment']]
df.head()

,review,sentiment
0,"A true masterpiece of our time. The visuals, t...",positive
1,Absolutely breathtaking. This isn't just a mov...,positive
2,Mind-bending and emotionally resonant. Interst...,positive
3,One of the best films I have ever seen. The sc...,positive
4,Pure cinematic genius. The practical effects a...,positive


In [13]:
import time
results = []

for review in df["review"]:
    try:
        res = analyze_review(review)

        if isinstance(res, list) and "args" in res[0]:
            res = res[0]["args"]

    except Exception as e:
        res = {"label": "ERROR", "confidence": 0.0, "explanation": str(e), "evident_phrases": []}
    
    results.append(res)
    time.sleep(2)
results_df = pd.DataFrame(results)
final_df = pd.concat([df, results_df], axis=1)

print("Final columns:", final_df.columns)
final_df.head()


Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjec

Final columns: Index(['review', 'sentiment', 'explanation', 'evident_phrases', 'confidence',
       'label'],
      dtype='object')


,review,sentiment,explanation,evident_phrases,confidence,label
0,"A true masterpiece of our time. The visuals, t...",positive,The review uses strong positive language like ...,"[true masterpiece of our time, everything is p...",0.98,Positive
1,Absolutely breathtaking. This isn't just a mov...,positive,The review uses overwhelmingly positive langua...,"[Absolutely breathtaking, it's an experience, ...",0.98,Positive
2,Mind-bending and emotionally resonant. Interst...,positive,The review uses highly positive language like ...,"[Mind-bending, emotionally resonant, stunning ...",0.95,Positive
3,One of the best films I have ever seen. The sc...,positive,The review uses strong positive language like ...,"[best films I have ever seen, career-best perf...",0.98,Positive
4,Pure cinematic genius. The practical effects a...,positive,The review uses strong positive language like ...,"[Pure cinematic genius, incredible, intelligen...",0.95,Positive


In [14]:
clean_df = final_df[final_df["label"] != "ERROR"].reset_index(drop=True)
clean_df.head()

,review,sentiment,explanation,evident_phrases,confidence,label
0,"A true masterpiece of our time. The visuals, t...",positive,The review uses strong positive language like ...,"[true masterpiece of our time, everything is p...",0.98,Positive
1,Absolutely breathtaking. This isn't just a mov...,positive,The review uses overwhelmingly positive langua...,"[Absolutely breathtaking, it's an experience, ...",0.98,Positive
2,Mind-bending and emotionally resonant. Interst...,positive,The review uses highly positive language like ...,"[Mind-bending, emotionally resonant, stunning ...",0.95,Positive
3,One of the best films I have ever seen. The sc...,positive,The review uses strong positive language like ...,"[best films I have ever seen, career-best perf...",0.98,Positive
4,Pure cinematic genius. The practical effects a...,positive,The review uses strong positive language like ...,"[Pure cinematic genius, incredible, intelligen...",0.95,Positive


In [15]:
clean_df.shape

(53, 6)

In [16]:
clean_df.to_csv("review_results.csv", index=False)

In [18]:
clean_df["label"].value_counts()

label
Positive    20
Negative    20
Neutral     13
Name: count, dtype: int64

In [23]:
accuracy = (clean_df["label"].str.lower() == clean_df["sentiment"]).mean()
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 90.57%


In [27]:
clean_df[clean_df["label"].str.lower() != clean_df["sentiment"]].head()

,review,sentiment,explanation,evident_phrases,confidence,label
38,"An interesting concept, though I felt the endi...",neutral,The reviewer found the concept interesting and...,"[ending was a bit too convenient, destination ...",0.75,Negative
40,"Worth watching for the spectacle alone, but do...",neutral,"The review suggests the movie is ""worth watchi...","[Worth watching for the spectacle alone, enter...",0.75,Positive
42,The performances are strong and the film is te...,neutral,The review highlights strong performances and ...,"[performances are strong, film is technically ...",0.75,Positive
48,The film is at its best when it's exploring th...,neutral,The review indicates a positive overall experi...,"[at its best, worthwhile, cinematic experience]",0.75,Positive
51,A solid movie that benefits from great perform...,neutral,The review highlights positive aspects like gr...,"[solid movie, great performances, amazing scor...",0.85,Positive
